In [ ]:
import os, math, random, pickle, re, numpy as np, time
from typing import List, Set, Tuple, Dict
from collections import Counter, defaultdict

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

DATA_DIR = "/content/drive/MyDrive/ML_Hackathon/Data"
CORPUS_PATH = os.path.join(DATA_DIR, "corpus.txt")
TEST_PATH   = os.path.join(DATA_DIR, "test.txt")
HMM_PATH    = os.path.join(DATA_DIR, "hangman_hmm.pkl")

os.makedirs(DATA_DIR, exist_ok=True)

ALPHABET  = [chr(c) for c in range(ord('a'), ord('z')+1)]
ALPH2IDX  = {ch:i for i,ch in enumerate(ALPHABET)}
IDX2ALPH  = {i:ch for ch,i in ALPH2IDX.items()}

print(f"Corpus Path: {CORPUS_PATH}")
print(f"Test Path:   {TEST_PATH}")
print(f"HMM Path:    {HMM_PATH}")
print("Google Drive paths verified.\n")

class LetterHMM:
    def __init__(self, alpha: float = 1.0):
        self.alpha = alpha
        self.pi = None
        self.A  = None
        self.trained = False

    def read_words(self, path: str) -> List[str]:
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            words = [re.sub(r'[^a-z]', '', w.strip().lower()) for w in f if w.strip()]
        return [w for w in words if w]

    def train(self, words: List[str]) -> None:
        init_cnt = Counter(); bigram_cnt = defaultdict(Counter)
        for w in words:
            if not w: continue
            init_cnt[w[0]] += 1
            for c1, c2 in zip(w[:-1], w[1:]):
                if c1 in ALPH2IDX and c2 in ALPH2IDX:
                    bigram_cnt[c1][c2] += 1
        V = len(ALPHABET); a = self.alpha
        self.pi = np.array([(init_cnt[ch]+a) for ch in ALPHABET], float)
        self.pi /= self.pi.sum()
        self.A = np.zeros((V,V))
        for i,p in enumerate(ALPHABET):
            tot = sum(bigram_cnt[p][ch] for ch in ALPHABET)+a*V
            for j,q in enumerate(ALPHABET):
                self.A[i,j]=(bigram_cnt[p][q]+a)/tot
        self.trained = True
        print("HMM trained.")

    def _emission_vector(self,ch,excluded):
        e=np.zeros(26)
        if ch=="_":
            avail=[a for a in ALPHABET if a not in excluded] or ALPHABET
            for a in avail: e[ALPH2IDX[a]]=1
        else:
            if ch in ALPH2IDX: e[ALPH2IDX[ch]]=1
            else: e[:]=1
        e/=e.sum(); return e

    def _build_emissions(self,pattern,guessed):
        wrong={g for g in guessed if g not in pattern}
        excluded={g for g in guessed if g in ALPH2IDX}
        E=np.zeros((len(pattern),26))
        for i,ch in enumerate(pattern.lower()):
            E[i]=self._emission_vector(ch,wrong|excluded)
        return E

    def _forward(self,E):
        T=E.shape[0]; a=np.zeros_like(E); c=np.zeros(T)
        a[0]=self.pi*E[0]; s=a[0].sum() or 1; a[0]/=s; c[0]=s
        for t in range(1,T):
            a[t]=(a[t-1]@self.A)*E[t]; s=a[t].sum() or 1; a[t]/=s; c[t]=s
        return a,c

    def _backward(self,E,c):
        T=E.shape[0]; b=np.zeros_like(E); b[-1]=1/26
        for t in range(T-2,-1,-1):
            b[t]=self.A@(E[t+1]*b[t+1]); s=b[t].sum() or 1; b[t]/=s
        return b

    def posteriors(self,pattern,guessed):
        E=self._build_emissions(pattern,guessed)
        a,c=self._forward(E); b=self._backward(E,c)
        g=a*b; g/=g.sum(axis=1,keepdims=True)
        return g

    def save(self,path):
        with open(path,"wb") as f:
            pickle.dump({'alpha':self.alpha,'pi':self.pi,'A':self.A,'trained':self.trained},f)

    @classmethod
    def load(cls,path):
        with open(path,"rb") as f: obj=pickle.load(f)
        m=cls(alpha=obj['alpha']); m.pi=obj['pi']; m.A=obj['A']; m.trained=obj['trained']; return m


# Load or train HMM
if os.path.exists(HMM_PATH):
    hmm = LetterHMM.load(HMM_PATH)
    print("Loaded HMM from Drive.")
else:
    hmm = LetterHMM()
    words = hmm.read_words(CORPUS_PATH)
    hmm.train(words)
    hmm.save(HMM_PATH)
    print("HMM model saved to Drive.")

train_words = hmm.read_words(CORPUS_PATH)
test_words  = hmm.read_words(TEST_PATH)
print(f"Loaded {len(train_words)} training words, {len(test_words)} test words.\n")

class HangmanEnv:
    def __init__(self,words,max_wrong=6):
        self.words=words; self.max_wrong=max_wrong; self.reset()
    def reset(self,word=None):
        self.word=random.choice(self.words) if word is None else word
        self.pattern=["_"]*len(self.word)
        self.guessed=set(); self.wrong=0; self.repeats=0; self.done=False
        return self._obs()
    def _obs(self): return ("".join(self.pattern),self.guessed.copy(),self.wrong,self.done)
    def step(self,letter):
        if self.done: return self._obs(),0,True,{}
        if letter in self.guessed:
            self.repeats+=1; return self._obs(),-2,False,{}
        self.guessed.add(letter)
        reward=0
        if letter in self.word:
            cnt=0
            for i,ch in enumerate(self.word):
                if ch==letter and self.pattern[i]=="_":
                    self.pattern[i]=letter; cnt+=1
            reward=8*cnt
        else: self.wrong+=1; reward=-6
        if "_" not in self.pattern:
            self.done=True; reward+=60
        elif self.wrong>=self.max_wrong:
            self.done=True; reward-=25
        return self._obs(),reward,self.done,{}

class SmartWordMatcher:
    def __init__(self,words):
        self.words_by_len=defaultdict(list)
        for w in words: self.words_by_len[len(w)].append(w)
    def get_letter_probs(self,pattern,guessed):
        matches=self.words_by_len.get(len(pattern),[])
        cand=[]; wrong={g for g in guessed if g not in pattern}
        for w in matches:
            if any(ch in w for ch in wrong): continue
            if all(p=="_" or p==w[i] for i,p in enumerate(pattern)): cand.append(w)
        freq=np.zeros(26)
        for w in cand:
            for ch in set(w):
                if ch not in guessed and ch in ALPH2IDX: freq[ALPH2IDX[ch]]+=1
        if freq.sum()==0: freq[:]=1
        freq/=freq.sum(); return freq

def hmm_vec(hmm,pattern,guessed):
    g=hmm.posteriors(pattern,guessed)
    v=np.zeros(26)
    for i,ch in enumerate(pattern):
        if ch=="_": v+=g[i]
    v=v/v.sum() if v.sum()>0 else np.ones(26)/26
    return v

class HybridPolicy:
    def __init__(self,words):
        self.matcher=SmartWordMatcher(words)
        self.theta=np.random.randn(26*4+4)*0.01
    def get_feats(self,pattern,guessed,hmm):
        g1=hmm_vec(hmm,pattern,guessed)
        g2=self.matcher.get_letter_probs(pattern,guessed)
        L=6-len([g for g in guessed if g not in pattern])
        b=pattern.count("_")
        prog=1-b/len(pattern) if len(pattern)>0 else 0
        rev=len(pattern)-b
        return g1,g2,L,b,prog,rev
    def compute_scores(self,g1,g2,L,b,prog,rev,mask):
        i=0; f1=self.theta[i:i+26]; i+=26
        f2=self.theta[i:i+26]; i+=26
        f3=self.theta[i:i+26]; i+=26
        f4=self.theta[i:i+26]; i+=26
        bL,bB,bP,bR=self.theta[i:i+4]
        mix=0.7*g1+0.3*g2
        s=f1*g1+f2*g2+f3*mix+f4*(g1+g2)/2
        s+=(bL*L+bB*b+bP*prog+bR*rev)
        s*=mask; s[mask==0]=-1e9
        return s
    def get_probs(self,scores,temp=1):
        scores=scores/temp; scores-=scores.max()
        p=np.exp(scores); p/=p.sum() if p.sum()>0 else 26
        return p
    def sample_action(self,pattern,guessed,hmm,temp=1):
        g1,g2,L,b,prog,rev=self.get_feats(pattern,guessed,hmm)
        mask=np.ones(26)
        for ch in guessed:
            if ch in ALPH2IDX: mask[ALPH2IDX[ch]]=0
        s=self.compute_scores(g1,g2,L,b,prog,rev,mask)
        p=self.get_probs(s,temp)
        a=np.random.choice(26,p=p)
        return a,p,(g1,g2,L,b,prog,rev,mask)
    def greedy_action(self,pattern,guessed,hmm):
        g1,g2,L,b,prog,rev=self.get_feats(pattern,guessed,hmm)
        mask=np.ones(26)
        for ch in guessed:
            if ch in ALPH2IDX: mask[ALPH2IDX[ch]]=0
        s=self.compute_scores(g1,g2,L,b,prog,rev,mask)
        return np.argmax(s)

def compute_grad(policy,traj,adv,_):
    g=np.zeros_like(policy.theta)
    for a,p,st in traj:
        g1,g2,L,b,prog,rev,mask=st
        d=np.zeros(26); d[a]=1; d-=p
        i=0
        g[i:i+26]+=adv*d*g1; i+=26
        g[i:i+26]+=adv*d*g2; i+=26
        g[i:i+26]+=adv*d*(0.7*g1+0.3*g2); i+=26
        g[i:i+26]+=adv*d*(g1+g2)/2; i+=26
        s=d.sum()
        g[i]+=adv*s*L; i+=1; g[i]+=adv*s*b; i+=1; g[i]+=adv*s*prog; i+=1; g[i]+=adv*s*rev
    return g

def entropy_of_probs(p): return -np.sum(p*np.log(np.clip(p,1e-12,1.0)))

def run_episode(env,policy,hmm,temp=1):
    traj=[]; env.reset(); total=0; ent=0
    for _ in range(60):
        pat,gu,w,d=env._obs()
        if d: break
        a,p,st=policy.sample_action(pat,gu,hmm,temp)
        ent+=entropy_of_probs(p)
        _,r,d,_=env.step(ALPHABET[a])
        total+=r; traj.append((a,p,st))
        if d: break
    total=float(np.clip(total,-200,200))
    return traj,total,ent,env

def evaluate_policy(policy,hmm,words,games=500):
    env=HangmanEnv(words); wins=wrong=rep=0
    for _ in range(games):
        env.reset()
        while True:
            pat,gu,w,d=env._obs()
            if d: break
            a=policy.greedy_action(pat,gu,hmm)
            _,_,d,_=env.step(ALPHABET[a])
            if d: break
        if "_" not in env.pattern: wins+=1
        wrong+=env.wrong; rep+=env.repeats
    succ=wins/games; score=(succ*2000)-(5*wrong)-(2*rep)
    return succ,score

def train_policy_improved(words,hmm,episodes=20000,lr=5e-4,entropy_coef=0.02,grad_clip=5.0,baseline_beta=0.995):
    policy=HybridPolicy(words); baseline=0; best=-1e9; best_theta=policy.theta.copy()
    env=HangmanEnv(words)
    for ep in range(1,episodes+1):
        temp=max(0.2,1.2-(ep/episodes))
        traj,G,ent,e=run_episode(env,policy,hmm,temp)
        adv=G-baseline; grad=compute_grad(policy,traj,adv,0)
        grad += entropy_coef*np.sign(grad)*ent
        gn=np.linalg.norm(grad)
        if gn>grad_clip: grad*=grad_clip/(gn+1e-12)
        policy.theta+=lr*grad
        baseline=baseline_beta*baseline+(1-baseline_beta)*G
        if G>best: best=G; best_theta=policy.theta.copy()
        if ep%1000==0:
            succ,score=evaluate_policy(policy,hmm,random.sample(words,min(500,len(words))))
            print(f"Ep {ep:5d} | R={G:7.2f} | Base={baseline:7.2f} | Succ={succ*100:5.2f}% | Score={score:7.1f}")
    policy.theta=best_theta
    return policy

print("Training improved policy...\n")
policy=train_policy_improved(train_words,hmm,episodes=10000,lr=5e-4,entropy_coef=0.02)

print("\nEvaluating on test set...")
succ,score=evaluate_policy(policy,hmm,test_words,games=2000)
print(f"\nFinal Success Rate: {succ*100:.2f}%")
print(f"Final Score: {score:.1f}\n")
